### Creacion de nuevas columnas en un dataFrame

#### Creacion de una columna con un valor constante

Si bien es muy poco usual hacer este tipo de cosas, puede ser util saber hacerlo para generar una columna con un determinado valor de default por poner un ejemplo tipico.

Como vemos en el ejemplo que sigue al setear el indice usamos el atributo inplace=True, esto es para que en la misma linea haga el seteo sin necesidad de asignar el resultado del mismo a la misma variable data, es un seteo con asignacion inline como se dice.

Lo del drop(['high', 'low'], axis=1) es simplemente borrar esas dos colunas para que me entre mejor en la pagina la tabla y limpiar un poco de datos que no necesito ver.


In [65]:
import pandas as pd
data = pd.read_excel('AAPL.xlsx', sheet_name='Hoja1')
data.set_index('timestamp', inplace=True)
data['Precio_COME']=3
data.drop(['high', 'low'], axis=1).head()

,open,close,adjusted_close,volume,Precio_COME
timestamp,,,,,
2020-03-06,282.00,289.03,289.03,56544246,3
2020-03-05,295.52,292.92,292.92,46893219,3
2020-03-04,296.44,302.74,302.74,54794568,3
2020-03-03,303.67,289.32,289.32,79868852,3
2020-03-02,282.28,298.81,298.81,85349339,3


Drop es borrar obviamente, el listado de columnas va entre corchetes porque es una lista, y luego uso axis=1 indicando que me refiero a columnas, ya que si quiero borrar filas uso axis=0 que es el valor default de la variable axis.

Tengamos en cuenta siempre lo siguiente, cuando escribimos en la Jupyter notebook:


In [66]:
data.drop(['high', 'low'], axis=1).head()

,open,close,adjusted_close,volume,Precio_COME
timestamp,,,,,
2020-03-06,282.00,289.03,289.03,56544246,3
2020-03-05,295.52,292.92,292.92,46893219,3
2020-03-04,296.44,302.74,302.74,54794568,3
2020-03-03,303.67,289.32,289.32,79868852,3
2020-03-02,282.28,298.81,298.81,85349339,3


Directamente me "da salida" y me imprime la tabla como vimos, pero si lo hacemos en un IDE sin poner print no nos devuelve nada... pero hay mas, al hacerlo sin haberle asignado esa salida a "data" si ahora le volvemos a pedir que imprima data van a aparecer las columnas borradas nuevamente, porque lo que hicimos fue solamente ver la salida de la linea pero no se lo asignamos a la variable, para ello deberiamos haber escrito:


In [67]:
data = data.drop(['high', 'low'], axis=1).head()
data

,open,close,adjusted_close,volume,Precio_COME
timestamp,,,,,
2020-03-06,282.00,289.03,289.03,56544246,3
2020-03-05,295.52,292.92,292.92,46893219,3
2020-03-04,296.44,302.74,302.74,54794568,3
2020-03-03,303.67,289.32,289.32,79868852,3
2020-03-02,282.28,298.81,298.81,85349339,3


#### Creacion de una columna a partir de otras columnas

Tambien se puede crear una columna nueva partiendo de los valores de otra u otras columnas. Veamos unos ejemplos.

Creamos una columna que muestre el precio medio entre apertura, cierre, maximo y minimo


In [68]:
data = pd.read_excel('AAPL.xlsx', sheet_name="Hoja1")
data.set_index('timestamp', inplace=True)

data['precio_medio'] = (data.open + data.close + data.low + data.high) /4
data.drop(['high', 'low'], axis=1).head()

,open,close,adjusted_close,volume,precio_medio
timestamp,,,,,
2020-03-06,282.00,289.03,289.03,56544246,285.7700
2020-03-05,295.52,292.92,292.92,46893219,294.8500
2020-03-04,296.44,302.74,302.74,54794568,298.9275
2020-03-03,303.67,289.32,289.32,79868852,295.6975
2020-03-02,282.28,298.81,298.81,85349339,290.0625


Armemos ahora una columna que muestre el volumen diario en millones de usd en lugar de nominales, usando el precio medio de cada rueda.


In [69]:
data = pd.read_excel('AAPL.xlsx', sheet_name="Hoja1")
data.set_index('timestamp', inplace=True)

data['precio_medio'] = (data.open + data.close + data.low + data.high) /4
data['vol_mln_usd'] = round((data.volume*data.precio_medio)/1000000)

data.drop(['high', 'low', 'open', 'close'], axis=1).head()

,adjusted_close,volume,precio_medio,vol_mln_usd
timestamp,,,,
2020-03-06,289.03,56544246,285.7700,16159.0
2020-03-05,292.92,46893219,294.8500,13826.0
2020-03-04,302.74,54794568,298.9275,16380.0
2020-03-03,289.32,79868852,295.6975,23617.0
2020-03-02,298.81,85349339,290.0625,24757.0


#### Creacion de una columna iterando el dataFrame

Si bien es un recurso muy usado, en realidad es poco eficiente cuando ya estamos trabajando con Pandas, pero por ahora no nos vamos a ocupar de la eficiencia, por lo general cualquier tipo de calculo que queramos hacer partiendo de datos de una columna para generar otra columna, se va a poder hacer con funciones especificas de Pandas que ya iremos viendo.

Veamos a modo de ejemplo como podria generar una nueva columna al dataFrame iterando el mismo.

Vamos a crear una columna que informe el color de la vela de cada rueda.

Recordemos que la vela es verde cuando el precio de cierre es mayor al de apertura y es roja cuando pasa lo opuesto.

En el ejemplo veran que dentro del bucle FOR nos referimos a "locaciones" .loc[fila, columna] ya lo veremos mas adelante en detalle.

Como nos daremos cuenta loc[] nos permite acceder a determinados lugares o locaciones del dataFrame, es como el equivalente a las celdas del Excel, cuando poniamos "B5" nos referiamos a la columna "B" y la fila "5", bueno, aca en pandas usaremos algo asi como .loc[5, "B"] "digamos"


In [70]:
data = pd.read_excel('AAPL.xlsx', sheet_name="Hoja1")
# data.set_index('timestamp', inplace=True) The error occurs because data.loc[i, 'close'] is trying to access rows using an integer index, but after setting timestamp as the index, the DataFrame is no longer indexed by integers but by timestamps.

for i in range(len(data)):
    if data.loc[i, 'close'] >= data.loc[i, 'open']:
        data.loc[i, 'color_vela'] = 'verde'
    else:
        data.loc[i, 'color_vela'] = 'roja'
        
data.drop(["high", "low"], axis=1).head(8)

,timestamp,open,close,adjusted_close,volume,color_vela
0,2020-03-06,282.00,289.03,289.03,56544246,verde
1,2020-03-05,295.52,292.92,292.92,46893219,roja
2,2020-03-04,296.44,302.74,302.74,54794568,verde
3,2020-03-03,303.67,289.32,289.32,79868852,roja
4,2020-03-02,282.28,298.81,298.81,85349339,verde
5,2020-02-28,257.26,273.36,273.36,106721230,verde
6,2020-02-27,281.10,273.52,273.52,80151381,roja
7,2020-02-26,286.53,292.65,292.65,49678431,verde


### Guardado de un DataFrame a un Excel

Vamos a agregar una columna a la tabla con la que venimos trabajando y luego guardar el dataFrame modificado, que como se imaginaran en lugar de read_excel sera algo parecido a "write_excel" y si, en este caso es "to_excel" el metodo en cuestion, y no mucha mas ciencia, pero veamoslo con un ejemplo rapido.

Primero vamos a generar una columna nueva, una manera rapida de hacerlo es tomar al dataFrame como si fuera un diccionario bidimensional y asignarle a la clave 'movimiento_diario' la diferencia entre el precio del cierre de cada dia menos el de apertura.


In [71]:
import pandas as pd
data=pd.read_excel('AAPL.xlsx', sheet_name="Hoja1")
data['mov_intra'] = data.close - data.open
data.drop(['high', 'low', 'adjusted_close'], axis=1).head()

,timestamp,open,close,volume,mov_intra
0,2020-03-06,282.00,289.03,56544246,7.03
1,2020-03-05,295.52,292.92,46893219,-2.60
2,2020-03-04,296.44,302.74,54794568,6.30
3,2020-03-03,303.67,289.32,79868852,-14.35
4,2020-03-02,282.28,298.81,85349339,16.53


No se si se sorprendieron o no de la simpleza con que Pandas nos permite trabajar matrices, la verdad que cuando lo vi me dejo con la boca abierta porque antes los programas no tenian tanta abstraccion de codigo como ahora y no "entendian" de forma tan sencilla lo que queriamos hacer.

Pandas esta preparado para trabajar con dataFrames, es decir, con tablas de datos, por lo que es obvio o presupone que cuando generemos un nuevo atributo lo vamos a querer para toda la tabla, asi que de forma tan sencilla como decirle resta lo que tiene en una columna y otra y meterlo en una tercera columna, la libreria ya lo entiende y procesa los datos internamente de la manera mas eficiente posible para que todo ese calculo e interacciones se haga con la mejor performance computacional, no se si lo notaron pero hace todo muy rapido, despues veremos ejemplos mas complejos igual.

Bueno, dicho esto entonces vamos a guardar en un Excel nuevo esta tabla con ese agregado que hicimos, para eso como anticipamos vamos a utilizar el metodo to_Excel de la libreria Pandas.


In [72]:
data.to_excel('AAPL_Modificado.xlsx', sheet_name='HojaEjemplo')